## Data Extraction from Database
Fill short gaps (e.g., less than 5 minutes) using linear interpolation

In [ ]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# Station code name mapping
station_mapping = {
    "16098828": "app_center",
    "16101230": "app_2pavlodar",
    "16101231": "app_pspu",
    "16101232": "app_metallurg",
    "16101233": "app_zaton"
}

# ====================== Helper Functions ======================

def prepare_station_data(df, station_name, tz='Asia/Tashkent', current=True):
    """
    Prepares data for selected station:
      - Converts dates and adds additional time columns,
      - Applies station mapping,
      - Filters data by station,
      - Localizes timestamps and, if current=True, updates last_record_date to current time.
    """
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['year']  = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day']   = df['date'].dt.day
    df['hour']  = df['date'].dt.hour
    df['minute']= df['date'].dt.minute
    df['station_name'] = df['app_id'].astype(str).map(station_mapping)
    station_data = df[df['station_name'] == station_name]
    
    first_record_date = station_data['date'].min()
    if first_record_date.tz is None:
        first_record_date = first_record_date.tz_localize(tz)
    last_record_date = station_data['date'].max()
    if last_record_date.tz is None:
        last_record_date = last_record_date.tz_localize(tz)
    
    if current:
        analysis_end = pd.Timestamp.now(tz=tz).floor('min')
        if analysis_end > last_record_date:
            last_record_date = analysis_end

    return station_data, first_record_date, last_record_date

def get_months_ordered(first_record_date, last_record_date):
    """
    Returns a list of months as datetime, starting from the first month (with adjustment if date is not 1st)
    to the first day of the month following last_record_date.
    """
    end_for_range = (last_record_date + pd.offsets.MonthBegin()).replace(day=1)
    months_range = pd.date_range(start=first_record_date.replace(day=1),
                                 end=end_for_range, freq="MS").to_pydatetime().tolist()
    if first_record_date.day != 1:
        months_range[0] = first_record_date
    return months_range

def plot_month_fill_common(ax, station_name, month_data, month_name, total_minutes_in_month, first_record_date, last_record_date, parameter=None):
    """
    Draws heatmap of data completeness for a month.
      - If parameter is None, considers presence of records,
      - Otherwise, if parameter value is not NaN – considers data present.
    """
    days_in_month = pd.Timestamp(month_name).days_in_month
    day_minutes = np.full((days_in_month, 24 * 60), 2)  # 2 = outside measurement period (white)
    tz = last_record_date.tz
    month_start = pd.Timestamp(f"{month_name}-01", tz=tz)
    month_end   = pd.Timestamp(f"{month_name}-{days_in_month}", tz=tz) + pd.Timedelta(hours=23, minutes=59)
    
    if first_record_date and last_record_date:
        if month_end < first_record_date or month_start > last_record_date:
            completeness_percentage = 0
        else:
            for _, row in month_data.iterrows():
                day_index = row['day'] - 1
                minute_index = row['hour'] * 60 + row['minute']
                if parameter is None:
                    day_minutes[day_index, minute_index] = 1
                else:
                    if pd.notna(row[parameter]):
                        day_minutes[day_index, minute_index] = 1

            for day in range(days_in_month):
                for minute in range(24 * 60):
                    current_datetime = pd.Timestamp(f"{month_name}-{day+1}", tz=tz) + pd.Timedelta(minutes=minute)
                    if first_record_date <= current_datetime <= last_record_date:
                        if day_minutes[day, minute] == 2:
                            day_minutes[day, minute] = 0

            completeness_percentage = np.sum(day_minutes == 1) / total_minutes_in_month * 100
    else:
        completeness_percentage = 0

    cmap = mcolors.ListedColormap(["red", "green", "white"])
    bounds = [0, 1, 2, 3]
    norm = mcolors.BoundaryNorm(bounds, cmap.N)
    
    ax.imshow(day_minutes, aspect='auto', cmap=cmap, norm=norm, interpolation='nearest')
    title_text = f"Data Completeness" if parameter is None else f"Completeness {parameter}"
    ax.set_title(f"{month_name} ({station_name})\n{title_text}: {completeness_percentage:.1f}%", fontsize=10)
    ax.set_xlabel("Hours of day")
    ax.set_ylabel("Day of month")
    ax.set_xticks(range(0, 24 * 60, 60))
    ax.set_xticklabels(range(0, 24))
    days_labels = [str(day) for day in range(1, days_in_month + 1)]
    ax.set_yticks(range(0, days_in_month))
    ax.set_yticklabels(days_labels)

# ============== Main Plotting Functions ==============

def plot_data_completeness(df, station_name, current=True):
    station_data, first_record_date, last_record_date = prepare_station_data(df, station_name, current=current)
    months_ordered = get_months_ordered(first_record_date, last_record_date)
    
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15))
    axes = axes.flatten()
    for i, month_datetime in enumerate(months_ordered):
        ax = axes[i]
        month_data = station_data[
            (station_data['year'] == month_datetime.year) &
            (station_data['month'] == month_datetime.month)
        ]
        days_in_month = pd.Timestamp(month_datetime).days_in_month
        total_minutes = days_in_month * 24 * 60
        month_name = month_datetime.strftime("%Y-%m")
        plot_month_fill_common(ax, station_name, month_data, month_name, total_minutes, first_record_date, last_record_date, parameter=None)
    for ax in axes[len(months_ordered):]:
        ax.axis("off")
    
    plt.tight_layout()
    plt.suptitle(f"Data Completeness for: {station_name}", fontsize=20)
    plt.subplots_adjust(top=0.9)
    plt.show()

def plot_param_completeness(df, station_name, parameter, current=True):
    station_data, first_record_date, last_record_date = prepare_station_data(df, station_name, current=current)
    months_ordered = get_months_ordered(first_record_date, last_record_date)
    
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15))
    axes = axes.flatten()
    for i, month_datetime in enumerate(months_ordered):
        ax = axes[i]
        month_data = station_data[
            (station_data['year'] == month_datetime.year) &
            (station_data['month'] == month_datetime.month)
        ]
        days_in_month = pd.Timestamp(month_datetime).days_in_month
        total_minutes = days_in_month * 24 * 60
        month_name = month_datetime.strftime("%Y-%m")
        plot_month_fill_common(ax, station_name, month_data, month_name, total_minutes, first_record_date, last_record_date, parameter=parameter)
    for ax in axes[len(months_ordered):]:
        ax.axis("off")
    
    plt.tight_layout()
    plt.suptitle(f"Completeness of {parameter} for station: {station_name}", fontsize=20)
    plt.subplots_adjust(top=0.9)
    plt.show()

In [ ]:
# Database connection
db_path = "air_quality_data.sqlite"
conn = sqlite3.connect(db_path)

# Set station
station = "16101231"

# Set dates with time.

# ---Research dates---
start_date = "2024-05-23 00:00:00"  # Or False for automatic substitution
end_date = "2025-01-19 23:59:59"      # Or False for automatic substitution
# ----------------------------

if start_date is False or end_date is False:
    if start_date is False:
        query_min = f"SELECT MIN(date) as min_date FROM air_quality WHERE app_id = {station}"
        start_date = pd.read_sql_query(query_min, conn).iloc[0]['min_date']
    if end_date is False:
        query_max = f"SELECT MAX(date) as max_date FROM air_quality WHERE app_id = {station}"
        end_date = pd.read_sql_query(query_max, conn).iloc[0]['max_date']

query = f"""
SELECT app_id, date, air_temperature, air_humidity, pm2_5 
FROM air_quality 
WHERE app_id = {station} 
  AND date BETWEEN '{start_date}' AND '{end_date}'
ORDER BY date
"""

df = pd.read_sql_query(query, conn)
conn.close()

print(f"Number of rows in dataframe: {len(df)}")
df['date'] = pd.to_datetime(df['date']).dt.floor('min')
# print(df.head())
print(df.tail())

# ====================== Function Call Examples ======================
# plot_data_completeness(df, station_mapping[station], current=False)
plot_param_completeness(df, station_mapping[station], "pm2_5", current=False)

## Data Outlier Detection

In [ ]:
print(df.columns)

df_init = df.copy()  # Create a copy to avoid modifying the original dataset

nan_rows_ = df_init[df_init["pm2_5"].isna()]
print(nan_rows_.head())

df_init = df_init.dropna(subset=["pm2_5"])  # Remove rows where pm2_5 == NaN

nan_rows_ = df_init[df_init["pm2_5"].isna()]
print(nan_rows_.head())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio

# Function for primary outlier detection
def primary_outlier_detection(df, column="pm2_5"):
    df["is_primary_outlier"] = (df[column] > 200) & (df[column] > 3 * df[column].shift(1))
    return df

# Function for outlier detection using IQR method
def detect_outliers_iqr(df, column="pm2_5"):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 10 * IQR
    upper_bound = Q3 + 10 * IQR

    # Outlier mask
    df["is_outlier"] = (df[column] < lower_bound) | (df[column] > upper_bound)

    return df, lower_bound, upper_bound

# Apply primary outlier detection method to data
df_init = primary_outlier_detection(df_init, column="pm2_5")

# Visualize primary outliers
fig = go.Figure()

# Line with all values
fig.add_trace(go.Scatter(
    x=df_init["date"], 
    y=df_init["pm2_5"],
    mode="lines",
    # name="All data",
    line=dict(color="blue")
))

# Primary outlier points
primary_outliers = df_init[df_init["is_primary_outlier"]]
fig.add_trace(go.Scatter(
    x=primary_outliers["date"], 
    y=primary_outliers["pm2_5"],
    mode="markers",
    # name="Primary outliers",
    marker=dict(color="red", size=6, symbol="circle-open")
))

# Configure axes and display
fig.update_layout(
    # title="Interactive PM2.5 plot with primary outliers",
    xaxis_title="Datetime",
    yaxis_title="PM2.5 (µg/m³)",
    xaxis=dict(rangeslider=dict(visible=True), type="date"),
    legend=dict(x=0, y=1)
)

# Display plot
fig.show()

# Remove primary outliers
df_no_primary_outliers = df_init[~df_init["is_primary_outlier"]]

# Visualize data without primary outliers
fig = go.Figure()

# Line without primary outliers
fig.add_trace(go.Scatter(
    x=df_no_primary_outliers["date"], 
    y=df_no_primary_outliers["pm2_5"],
    mode="lines",
    # name="Data without primary outliers",
    line=dict(color="blue")
))

# Configure axes and display
fig.update_layout(
    # title="Interactive PM2.5 plot without primary outliers",
    xaxis_title="Datetime",
    yaxis_title="PM2.5 (µg/m³)",
    xaxis=dict(rangeslider=dict(visible=True), type="date"),
    legend=dict(x=0, y=1)
)

# Display plot
fig.show()

# Apply IQR method to data without primary outliers
df_no_primary_outliers, lower_bound, upper_bound = detect_outliers_iqr(df_no_primary_outliers, column="pm2_5")

# Visualize outliers after IQR method
fig = go.Figure()

# Line with all values after primary screening
fig.add_trace(go.Scatter(
    x=df_no_primary_outliers["date"], 
    y=df_no_primary_outliers["pm2_5"],
    mode="lines",
    name="All data",
    line=dict(color="blue")
))

# Outlier points after IQR method
iqr_outliers = df_no_primary_outliers[df_no_primary_outliers["is_outlier"]]
fig.add_trace(go.Scatter(
    x=iqr_outliers["date"], 
    y=iqr_outliers["pm2_5"],
    mode="markers",
    name="Outliers (after IQR)",
    marker=dict(color="red", size=6, symbol="circle-open")
))

# Configure axes and display
fig.update_layout(
    # title="Interactive PM2.5 plot with outliers (IQR method)",
    xaxis_title="Datetime",
    yaxis_title="PM2.5 (µg/m³)",
    xaxis=dict(rangeslider=dict(visible=False), type="date"),
    legend=dict(
        x=1,
        y=1,
        xanchor="right",
        yanchor="top"
    )
)

# Display plot
fig.show()

# Save plot in high resolution
# pio.write_image(fig, 'high_res_plot.png', width=1000, height=600, scale=6)

# Remove outliers after IQR method
df_no_outliers = df_no_primary_outliers[~df_no_primary_outliers["is_outlier"]]

# Visualize data without outliers after IQR method
fig = go.Figure()

# Line without outliers after IQR method
fig.add_trace(go.Scatter(
    x=df_no_outliers["date"], 
    y=df_no_outliers["pm2_5"],
    mode="lines",
    # name="Data without outliers (IQR method)",
    line=dict(color="blue")
))

# Configure axes and display
fig.update_layout(
    # title="Interactive PM2.5 plot without outliers (IQR method)",
    xaxis_title="Datetime",
    yaxis_title="PM2.5 (µg/m³)",
    xaxis=dict(rangeslider=dict(visible=False), type="date"),
    legend=dict(x=0, y=1)
)

# Display plot
fig.show()

In [ ]:
# Replace outliers with NaN instead of removing rows
df_cleaned = df_no_outliers.copy()
df_cleaned.loc[df_cleaned["is_outlier"], "pm2_5"] = np.nan
df_cleaned.loc[df_cleaned["is_primary_outlier"], "pm2_5"] = np.nan
# Remove helper column "is_outlier" if no longer needed
df_cleaned = df_cleaned.drop(columns=["is_outlier"])
df_cleaned = df_cleaned.drop(columns=["is_primary_outlier"])

# Output number of rows before and after outlier processing
print(f"Number of rows BEFORE outlier processing: {len(df_no_outliers)}")
print(f"Number of rows AFTER outlier processing (should be the same): {len(df_cleaned)}")

# Check that outliers were replaced with NaN
print(df_cleaned.head())
print(f"Number of NaN in 'pm2_5' after outlier replacement: {df_cleaned['pm2_5'].isna().sum()}")


## Fill Missing Minutes and Round Time to Minutes

In [ ]:

def fill_missing_minutes(df, time_column="date", station_column="app_id", parameters=None):
    """
    Fills missing minutes in data, adding them with NaN values for parameters.
    Automatically adds columns year, month, day, hour, minute and station_name.

    :param df: Original DataFrame.
    :param time_column: Name of date and time column.
    :param station_column: Name of station identifier column (e.g., "app_id").
    :param parameters: List of parameters to fill (if None, uses all numeric columns).
    :return: DataFrame with filled minutes.
    """
    df = df.copy()  # Avoid modifying the original dataframe
    df[time_column] = pd.to_datetime(df[time_column])  # Convert time to datetime
    df[station_column] = df[station_column].astype(str)  # Convert "app_id" to string

    if parameters is None:
        parameters = df.select_dtypes(include=[float, int]).columns.tolist()

    # Determine minimum and maximum date in data
    min_date, max_date = df[time_column].min(), df[time_column].max()

    # Create full time range with 1-minute step
    full_time_index = pd.date_range(start=min_date, end=max_date, freq="min")

    # Create new dataframe with all stations and full time index
    all_stations = df[station_column].unique()  # Take unique app_id (already strings)
    full_index = pd.MultiIndex.from_product([all_stations, full_time_index], names=[station_column, time_column])
    filled_df = pd.DataFrame(index=full_index).reset_index()

    # Merge with original data, filling missing rows with NaN
    df = pd.merge(filled_df, df, on=[station_column, time_column], how="left")

    # **Automatically add time components**
    df["year"] = df[time_column].dt.year
    df["month"] = df[time_column].dt.month
    df["day"] = df[time_column].dt.day
    df["hour"] = df[time_column].dt.hour
    df["minute"] = df[time_column].dt.minute

    # **Automatically add station_name**
    station_mapping = {
        "16098828": "app_center",
        "16101230": "app_2pavlodar",
        "16101231": "app_pspu",
        "16101232": "app_metallurg",
        "16101233": "app_zaton"
    }
    df["station_name"] = df[station_column].map(station_mapping)

    return df

df_filled = df_cleaned.copy()  # Create copy to avoid changing original data

print(df_filled.head())

plot_param_completeness(df_filled, station_name=station_mapping[station], parameter="pm2_5", current=False)

# Apply function to dataframe. Fill missing minutes in data, adding them with NaN values for parameters.
df_filled = fill_missing_minutes(df_filled, time_column="date", station_column="app_id", parameters=["pm2_5", "air_temperature",  "air_humidity"])

# Output first rows for verification
print(df_filled.head())

print(f"Number of rows in dataframe: {len(df_filled)}")

nan_rows = df_filled[df_filled["pm2_5"].isna()]
print(nan_rows.head())


In [ ]:
import missingno as msno
msno.matrix(df_filled)



## Correct Artificial Gaps (Due to Delayed Data Recording)

In [ ]:
def correct_late_measurements(df, time_column="date", station_column="station_name", threshold_seconds=15):
    """
    Corrects data delays when measurements are recorded with delay in the next minute.
    
    :param df: Original DataFrame.
    :param time_column: Name of date column.
    :param station_column: Name of station identifier column.
    :param threshold_seconds: Maximum number of seconds to assign measurement to previous minute.
    :return: DataFrame with corrected minutes.
    """

    df = df.copy()
    df["original_date"] = df[time_column]  # Save original date
    df["rounded_date"] = df[time_column].dt.floor("min")  # Round to minutes
    df["seconds"] = df[time_column].dt.second  # Extract seconds

    # Sort data by station and time
    df = df.sort_values(by=[station_column, time_column])

    # Create list for new records
    corrected_rows = []

    for station in df[station_column].unique():
        station_data = df[df[station_column] == station].copy()
        station_data = station_data.sort_values(by=time_column)

        # Go through each row
        for i in range(1, len(station_data)):
            prev_row = station_data.iloc[i - 1]
            curr_row = station_data.iloc[i]

            # If previous and current record have the same minute
            if prev_row["rounded_date"] == curr_row["rounded_date"]:
                continue  # Duplicates of same minute (skip)

            # If difference between records is more than 1 minute — potential gap
            diff_minutes = (curr_row["rounded_date"] - prev_row["rounded_date"]).seconds // 60

            if diff_minutes > 1 and curr_row["seconds"] < threshold_seconds:
                # Record new row belonging to missing minute
                missing_time = prev_row["rounded_date"] + pd.Timedelta(minutes=1)
                corrected_row = curr_row.copy()
                corrected_row["rounded_date"] = missing_time
                corrected_row["corrected"] = True  # Mark as corrected value
                corrected_rows.append(corrected_row)

    # Add corrected rows
    if corrected_rows:
        corrected_df = pd.DataFrame(corrected_rows)
        df = pd.concat([df, corrected_df], ignore_index=True)

    # Clean duplicates after corrections
    df = df.drop_duplicates(subset=[station_column, "rounded_date"], keep="first")

    # Return corrected dataframe with updated time_column
    df[time_column] = df["rounded_date"]
    df = df.drop(columns=["rounded_date", "seconds"], errors="ignore")
    return df

df_corrected_data = df_filled.copy()

# Count duplicates by station_name and date
def count_duplicates(df, time_column="date", station_column="station_name"):
    return df.duplicated(subset=[station_column, time_column], keep=False).sum()

# Output number of duplicates before correction
print(f"Number of time duplicates BEFORE correction: {count_duplicates(df_corrected_data)}")

# Apply correction
df_corrected = correct_late_measurements(df_corrected_data)

# Output number of duplicates after correction
print(f"Number of time duplicates AFTER correction: {count_duplicates(df_corrected)}")

print(df_corrected.head())

In [ ]:
def plot_param_over_time(df, param = "pm2_5"):
    """
    Plots PM2.5 changes over time.

    :param df: DataFrame with 'date' and 'pm2_5' columns
    """
    plt.figure(figsize=(20, 6))
    plt.plot(df["date"], df[param], marker="o", linestyle="", color="blue", markersize=3, label = param)

    plt.xlabel("Date and time")
    plt.ylabel(f"{param} concentration")
    plt.title(f"{param} changes over time (after outlier removal)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid()

    plt.show()

# Function call for visualization
plot_param_over_time(df_corrected, param = "pm2_5")

In [ ]:
def detect_gaps(df, time_column="date", value_column="pm2_5", station_column="station_name"):
    """
    Detects gaps in data for specified parameter.

    :param df: DataFrame with data
    :param time_column: Name of timestamp column
    :param value_column: Name of analyzed parameter column
    :param station_column: Name of station identifier column
    :return: DataFrame with gap characteristics (start_time, end_time, duration_minutes, gap_type)
    """
    df = df.copy()
    df[time_column] = pd.to_datetime(df[time_column])

    # Filter only needed columns and sort
    df = df[[station_column, time_column, value_column]].sort_values(by=[station_column, time_column])

    # Remove time duplicates
    df = df.drop_duplicates(subset=[station_column, time_column])

    # Define rows with NaN
    df["is_nan"] = df[value_column].isna()

    # Define gap start and end
    df["gap_start"] = df["is_nan"] & ~df["is_nan"].shift(1, fill_value=False)
    df["gap_end"] = df["is_nan"] & ~df["is_nan"].shift(-1, fill_value=False)

    # Form lists of start and end points of gaps
    gap_starts = df[df["gap_start"]][[station_column, time_column]].rename(columns={time_column: "start_time"})
    gap_ends = df[df["gap_end"]][[station_column, time_column]].rename(columns={time_column: "end_time"})

    # Ensure equal number of start and end points
    if len(gap_starts) != len(gap_ends):
        print("⚠️ Detected mismatch in number of gap start and end points!")

    # Combine gap start and end using indices
    gaps = pd.concat([gap_starts.reset_index(drop=True), gap_ends.reset_index(drop=True)], axis=1)

    # Calculate gap duration
    gaps["duration_minutes"] = (gaps["end_time"] - gaps["start_time"]).dt.total_seconds() / 60 + 1

    # Remove gaps with 0 minutes duration (if start_time == end_time)
    gaps = gaps[gaps["duration_minutes"] > 0]

    # Categorize gaps by duration
    bins = [0, 5, 30, np.inf]
    labels = ["short", "medium", "long"]
    gaps["gap_type"] = pd.cut(gaps["duration_minutes"], bins=bins, labels=labels, right=False)

    # Create index without duplicates for mapping neighboring values
    df_unique = df.drop_duplicates(subset=[time_column]).set_index(time_column)

    # Add neighboring values before and after gap
    gaps["before_gap"] = gaps["start_time"].map(df_unique[value_column].ffill())
    gaps["after_gap"] = gaps["end_time"].map(df_unique[value_column].bfill())

    return gaps.reset_index(drop=True)

gap_df_pm2_5 = detect_gaps(df_corrected, time_column="date", value_column="pm2_5", station_column="station_name")

gap_df_air_t = detect_gaps(df_corrected, time_column="date", value_column="air_temperature", station_column="station_name")

gap_df_air_h = detect_gaps(df_corrected, time_column="date", value_column="air_humidity", station_column="station_name")

# Output first 10 gaps
print(gap_df_pm2_5.head())  # Output first rows

# df_corrected[df_corrected.duplicated(subset=["date"], keep=False)]

# Check - output gaps with zero duration
# print(gap_df[gap_df["duration_minutes"] == 0])

## Linear Interpolation of Short Gaps (up to 5 minutes)

In [ ]:
def interpolate_short_gaps(df, time_column="date", value_column="pm2_5", max_gap=5):
    """
    Interpolates only short-term gaps (no more than max_gap minutes).

    :param df: DataFrame with time data.
    :param time_column: Name of date column.
    :param value_column: Name of parameter to fill gaps in.
    :param max_gap: Maximum gap length (in minutes) for interpolation.
    :return: DataFrame with partially filled gaps.
    """
    df = df.copy()
    df[time_column] = pd.to_datetime(df[time_column])  # Ensure datetime format

    # Define gaps
    df["is_nan"] = df[value_column].isna()

    # Define gap start and end
    df["gap_start"] = df["is_nan"] & ~df["is_nan"].shift(1, fill_value=False)
    df["gap_end"] = df["is_nan"] & ~df["is_nan"].shift(-1, fill_value=False)

    # Form lists of start and end points of gaps
    gap_starts = df[df["gap_start"]][[time_column]].rename(columns={time_column: "start_time"})
    gap_ends = df[df["gap_end"]][[time_column]].rename(columns={time_column: "end_time"})

    # Combine gap start and end
    gaps = pd.concat([gap_starts.reset_index(drop=True), gap_ends.reset_index(drop=True)], axis=1)

    # Calculate gap duration in minutes
    gaps["duration_minutes"] = (gaps["end_time"] - gaps["start_time"]).dt.total_seconds() / 60 + 1

    # Keep only short gaps (≤ max_gap minutes)
    short_gaps = gaps[gaps["duration_minutes"] <= max_gap]

    # Interpolate only short gaps
    for _, row in short_gaps.iterrows():
        start, end = row["start_time"], row["end_time"]
        mask = (df[time_column] >= start) & (df[time_column] <= end)
        df.loc[mask, value_column] = df[value_column].interpolate(method="linear", limit_direction="both")

    # Remove helper columns
    df = df.drop(columns=["is_nan", "gap_start", "gap_end"], errors="ignore")

    return df

# Fast interpolation function for only short gaps (up to max_gap minutes) with guaranteed capture of all minutes

def fast_interpolate_short_gaps(df, gap_df, time_column="date", value_column="pm2_5", max_gap=5):
    """
    Fast vectorized interpolation of short-term gaps with capture of all minutes inside gaps.
    """
    df = df.copy()
    df[time_column] = pd.to_datetime(df[time_column])

    # Select only short gaps
    short_gaps = gap_df[gap_df["duration_minutes"] <= max_gap]

    # Form **vectorized** mask for all rows falling into found gaps
    mask = df[time_column].apply(lambda t: any((t >= row["start_time"]) & (t <= row["end_time"]) for _, row in short_gaps.iterrows()))

    # Apply interpolation to found gaps
    df.loc[mask, value_column] = df[value_column].interpolate(method="linear", limit_direction="both")

    return df

In [ ]:
# Create copy of original dataframe
df_interp = df_corrected.copy()

# 1. Output number of gaps BEFORE interpolation
for param in ["pm2_5", "air_temperature", "air_humidity"]:
    print(f"Gaps in '{param}' BEFORE interpolation:", df_interp[param].isna().sum())

# 2. Visualize data completeness before interpolation
# plot_param_completeness(df_filled, station_name="app_pspu", parameter="pm2_5")

# 3. Apply interpolation only to short gaps
for param in ["pm2_5", "air_temperature", "air_humidity"]:
    df_interp = interpolate_short_gaps(df_interp, value_column=param, max_gap=5)

# 4. Output number of gaps AFTER interpolation
print("\nResults after interpolation:")
for param in ["pm2_5", "air_temperature", "air_humidity"]:
    print(f"Gaps in '{param}' AFTER interpolation:", df_interp[param].isna().sum())

In [ ]:
msno.matrix(df_interp)

print(df_interp.head())

In [ ]:
def plot_gap_visualization(df, gap_df, gap_index=0, param="pm2_5", wingspan=10):
    """
    Visualizes extracted gap in data, showing 10 points before and after the gap.
    
    :param df: DataFrame with data
    :param gap_df: DataFrame with detected gaps (must contain start_time and end_time)
    :param gap_index: Gap index from gap_df to visualize
    :param param: Parameter name for analysis (e.g., "pm2_5")
    """

    if gap_df.empty:
        print("No data in gaps table!")
        return
    
    # Get gap data
    first_gap = gap_df.iloc[gap_index]  # Gap by specified index
    start_time, end_time, duration = first_gap["start_time"], first_gap["end_time"], first_gap["duration_minutes"]

    # Output gap length information
    print(f"📌 Gap length - {duration} minutes (from {start_time} to {end_time})")

    # Select 10 points before and after gap
    before_gap = df[df["date"] < start_time].tail(wingspan)  # 10 points BEFORE gap
    after_gap = df[df["date"] > end_time].head(wingspan)  # 10 points AFTER gap
    gap_points = df[(df["date"] >= start_time) & (df["date"] <= end_time)]  # The gap itself
    
    # Combine data for lines
    combined_df = pd.concat([before_gap, gap_points, after_gap]).sort_values(by="date")

    # Plot construction
    plt.figure(figsize=(10, 5))
    
    # Line connects all points, including missing ones
    plt.plot(combined_df["date"], combined_df[param], linestyle="-", color="gray", alpha=0.5)

    # Original points (blue)
    plt.scatter(before_gap["date"], before_gap[param], color="blue", label="Original points")
    plt.scatter(after_gap["date"], after_gap[param], color="blue")

    # Filled points (red)
    plt.scatter(gap_points["date"], gap_points[param], color="red", label="Filled points")

    # Plot formatting
    plt.xlabel("Date and time")
    plt.ylabel(param)
    plt.title(f"Gap #{gap_index} visualization ({param})")
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid()

    # Show plot
    plt.show()

# plot_gap_visualization(df_interp, gap_df_pm2_5, gap_index=1, param="pm2_5", wingspan=20)

plot_gap_visualization(df_interp, gap_df_air_t, gap_index=46, param="air_temperature", wingspan=30) # example of wide gap gap_index=46

## Hourly Data Aggregation
Average PM2.5, temperature and humidity within each hour.
Check that each hour has sufficient data (≥40 minutes).
If data is insufficient, the hourly interval remains empty.

In [ ]:
import pandas as pd
import numpy as np

df_for_agregation = df_interp.copy()

# Ensure 'date' is in datetime format
df_for_agregation["date"] = pd.to_datetime(df_for_agregation["date"])

# Set 'date' as index
df_for_agregation.set_index("date", inplace=True)

# Parameters for averaging
params = ["pm2_5", "air_temperature", "air_humidity"]
min_count = 40  # Minimum number of points per hour

# Create empty DataFrame for hourly data with time axis
df_hourly = pd.DataFrame(index=df_for_agregation.resample("h").mean(numeric_only=True).index)

# Average by each parameter considering minimum number of measurements
for param in params:
    hourly_counts = df_for_agregation.resample("h")[param].count()  # Number of points in each hour
    valid_hours = hourly_counts[hourly_counts >= min_count].index  # Only hours with ≥40 measurements
    
    # Average and leave NaN where there's insufficient data
    df_hourly[param] = df_for_agregation.resample("h")[param].mean(numeric_only=True)
    df_hourly.loc[~df_hourly.index.isin(valid_hours), param] = np.nan

# Reset index, return 'date' as column
df_hourly.reset_index(inplace=True)

# Output result
from IPython.display import display
display(df_hourly)

print(df_hourly.head(75))



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming df_hourly is already created and contains "date" and "pm2_5" columns
# Sort data by date and create copy for work
df_out = df_hourly.copy().sort_values("date").reset_index(drop=True)

# Drop outliers for cleaner analysis (optional)
df_clean = df_out[~df_out["pm2_5"].isna()]  # remove rows with NaN in pm2.5

percentile = (df_clean["pm2_5"] <= 270).mean() * 100
print(f"Value 270 µg/m³ corresponds to {percentile:.2f}th percentile")
# Can also calculate other distribution characteristics
p_99 = np.percentile(df_clean["pm2_5"], 99)
p_99_6 = np.percentile(df_clean["pm2_5"], 99.6)
p_99_9 = np.percentile(df_clean["pm2_5"], 99.9)

print(f"99th percentile: {p_99:.2f} µg/m³")
print(f"99.6th percentile: {p_99_6:.2f} µg/m³")
print(f"99.9th percentile: {p_99_9:.2f} µg/m³")

# Initialize column for outlier marking
df_out["is_primary_outlier"] = False

# Primary outlier screening logic:
# A point is considered an outlier if:
# 1. Its value is greater than 270, or
# 2. Its value > 200 and more than 3 times higher than the previous point value.
# If subsequent points follow that satisfy the condition:
#    - value greater than 270, or
#    - value > 200 and doesn't drop sharply (not below 90% of previous value from group),
# then they are also marked as outliers until the first sharp drop.
N = len(df_out)
i = 1
while i < N:
    current_val = df_out.loc[i, "pm2_5"]
    previous_val = df_out.loc[i-1, "pm2_5"]
    
    # Check condition for detecting first outlier in group
    if (current_val > 270) or ((current_val > 200) and (current_val > 3 * previous_val)):
        df_out.loc[i, "is_primary_outlier"] = True
        group_val = current_val
        j = i + 1
        # Mark subsequent points while they satisfy conditions
        while j < N:
            next_val = df_out.loc[j, "pm2_5"]
            if (next_val > 270) or ((next_val > 200) and (next_val >= 0.9 * group_val)):
                df_out.loc[j, "is_primary_outlier"] = True
                group_val = next_val  # update group value if next point is greater or approximately equal to previous from group
                j += 1
            else:
                break
        i = j  # move to next point after outlier group
    else:
        i += 1

# Build first plot: all points blue, outliers (marked) red
plt.figure(figsize=(12, 6))
plt.plot(df_out["date"], df_out["pm2_5"], color="blue", label="All data")
plt.scatter(df_out.loc[df_out["is_primary_outlier"], "date"],
            df_out.loc[df_out["is_primary_outlier"], "pm2_5"],
            color="red", label="Outliers", zorder=5)
plt.xlabel("Datetime")
plt.ylabel("PM2.5 (µg/m³)")
# plt.title("PM2.5 plot with primary outlier screening")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join("output_diagrams", f"secondary_outliers_hourly.png"), dpi=600)
plt.show()


# Create DataFrame only with points not marked as outliers
df_hourly_no_outliers = df_out[~df_out["is_primary_outlier"]]

# Build second plot: only remaining points (without outliers)
plt.figure(figsize=(12, 6))
plt.plot(df_hourly_no_outliers["date"], df_hourly_no_outliers["pm2_5"], color="blue", label="Data without outliers")
plt.xlabel("Date")
plt.ylabel("PM2.5")
plt.title("PM2.5 plot after primary outlier removal")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def plot_hourly_param_completeness(df_hourly, parameter="pm2_5"):
    """
    Visualization of hourly data completeness for specified parameter.

    :param df_hourly: DataFrame with hourly average values.
    :param parameter: Parameter name (e.g., "pm2_5", "air_temperature", "air_humidity").
    """
    df_hourly["date"] = pd.to_datetime(df_hourly["date"])  # Ensure datetime format

    # Define full range of monitoring months
    first_record_date = df_hourly["date"].min()
    last_record_date = df_hourly["date"].max()

    # Ensure month boundaries are considered
    first_record_month_start = first_record_date.replace(day=1, hour=0, minute=0, second=0)
    last_record_month_start = last_record_date.replace(day=1, hour=0, minute=0, second=0)

    # Generate full list of months
    month_range = pd.date_range(start=first_record_month_start, end=last_record_month_start, freq="MS").to_pydatetime().tolist()

    # Function for plotting monthly data completeness
    def plot_month_fill(ax, month_data, month_name, first_record_date, last_record_date):
        days_in_month = pd.Timestamp(month_name).days_in_month
        hours_in_month = days_in_month * 24
        hourly_completeness = np.full(hours_in_month, 2)  # Default 2 (outside measurement range)

        # Define month boundaries
        month_start = pd.Timestamp(f"{month_name}-01")
        month_end = month_start + pd.Timedelta(days=days_in_month, hours=23)

        if first_record_date and last_record_date:
            if month_end < first_record_date or month_start > last_record_date:
                completeness_percentage = 0
            else:
                # Fill array (1 - data present, 0 - no data)
                for _, row in month_data.iterrows():
                    day_index = row["date"].day - 1
                    hour_index = row["date"].hour
                    index = day_index * 24 + hour_index
                    if pd.notna(row[parameter]):
                        hourly_completeness[index] = 1  # Data present

                # Update color map for hours within measurement period
                for hour in range(hours_in_month):
                    current_datetime = month_start + pd.Timedelta(hours=hour)
                    if first_record_date <= current_datetime <= last_record_date:
                        if hourly_completeness[hour] == 2:
                            hourly_completeness[hour] = 0  # No data
                    elif current_datetime < first_record_date:
                        hourly_completeness[hour] = 2  # Outside measurement period (white)

                # Calculate completeness percentage
                completeness_percentage = np.sum(hourly_completeness == 1) / hours_in_month * 100
        else:
            completeness_percentage = 0  # If no data at all

        # Create color map
        cmap = mcolors.ListedColormap(["red", "green", "white"])
        bounds = [0, 1, 2, 3]
        norm = mcolors.BoundaryNorm(bounds, cmap.N)

        # Build heatmap
        ax.imshow(hourly_completeness.reshape((days_in_month, 24)), aspect="auto", cmap=cmap, norm=norm, interpolation="nearest")
        ax.set_title(f"{month_name} ({parameter})\ncompleteness: {completeness_percentage:.1f}%", fontsize=10)
        ax.set_xlabel("Hours")
        ax.set_ylabel("Day of the month")

        # Configure X axis (hours)
        ax.set_xticks(np.arange(0, 24))  # Main hour marks
        ax.set_xticklabels(range(0, 24))  # Hours 0–23
        ax.set_xticks(np.arange(-0.5, 24, 1), minor=True)  # Intermediate divisions between hours

        # Configure Y axis (days of month)
        days_labels = [f"{day}" for day in range(1, days_in_month + 1)]
        ax.set_yticks(range(0, days_in_month))  # Main day marks
        ax.set_yticklabels(days_labels)
        ax.set_yticks(np.arange(-0.5, days_in_month, 1), minor=True)  # Intermediate divisions between days

        # Enable grid **only on intermediate divisions**
        ax.grid(which="minor", linestyle=":", linewidth=0.5, alpha=0.7)  # Keep only grid on intermediate divisions

    # Build calendar
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15))
    axes = axes.flatten()

    for i, month_datetime in enumerate(month_range):
        ax = axes[i]
        month_data = df_hourly[
            (df_hourly["date"].dt.year == month_datetime.year) &
            (df_hourly["date"].dt.month == month_datetime.month)
        ]
        month_name = month_datetime.strftime("%Y-%m")
        plot_month_fill(ax, month_data, month_name, first_record_date, last_record_date)

    # Remove extra axes
    for ax in axes[len(month_range):]:
        ax.axis("off")

    plt.tight_layout()
    # plt.suptitle(f"Hourly data completeness, parameter: {parameter}", fontsize=20)
    plt.subplots_adjust(top=0.9)
    plt.savefig(os.path.join("output_diagrams", f"hourly_data_completeness.png"), dpi=600)
    plt.show()


In [ ]:
plot_hourly_param_completeness(df_hourly_no_outliers, parameter="pm2_5")
# plot_hourly_param_completeness(df_hourly, parameter="air_temperature")
# plot_hourly_param_completeness(df_hourly, parameter="air_humidity")

In [ ]:
def detect_hourly_gaps(df, time_column="date", value_column="pm2_5"):
    """
    Detects gaps in hourly DataFrame for specified parameter.

    :param df: DataFrame with hourly data.
    :param time_column: Name of timestamp column.
    :param value_column: Name of analyzed parameter column.
    :return: DataFrame with gap characteristics (start_time, end_time, duration_hours, gap_type).
    """
    df = df.copy()
    df[time_column] = pd.to_datetime(df[time_column])  # Ensure datetime format

    # Filter needed columns and sort
    df = df[[time_column, value_column]].sort_values(by=time_column)

    # Remove time duplicates
    df = df.drop_duplicates(subset=[time_column])

    # Define rows with NaN
    df["is_nan"] = df[value_column].isna()

    # Define gap start and end
    df["gap_start"] = df["is_nan"] & ~df["is_nan"].shift(1, fill_value=False)
    df["gap_end"] = df["is_nan"] & ~df["is_nan"].shift(-1, fill_value=False)

    # Form lists of start and end points of gaps
    gap_starts = df[df["gap_start"]][[time_column]].rename(columns={time_column: "start_time"})
    gap_ends = df[df["gap_end"]][[time_column]].rename(columns={time_column: "end_time"})

    # Ensure equal number of start and end points
    if len(gap_starts) != len(gap_ends):
        print("⚠️ Detected mismatch in number of gap start and end points!")

    # Combine gap start and end
    gaps = pd.concat([gap_starts.reset_index(drop=True), gap_ends.reset_index(drop=True)], axis=1)

    # Calculate gap duration in hours (add 1 hour to include both ends)
    gaps["duration_hours"] = (gaps["end_time"] - gaps["start_time"]).dt.total_seconds() / 3600 + 1

    # Remove gaps with 0 hours duration (if start_time == end_time, they will be 1 hour)
    gaps = gaps[gaps["duration_hours"] > 0]

    # Categorize gaps by duration
    bins = [0, 2, 12, np.inf]  # <= 2 hours - short, <= 12 - medium, > 12 - long
    labels = ["short", "medium", "long"]
    gaps["gap_type"] = pd.cut(gaps["duration_hours"], bins=bins, labels=labels, right=False)

    # Create index without duplicates for mapping neighboring values
    df_unique = df.drop_duplicates(subset=[time_column]).set_index(time_column)

    # Add neighboring values before and after gap
    gaps["before_gap"] = gaps["start_time"].map(df_unique[value_column].ffill())
    gaps["after_gap"] = gaps["end_time"].map(df_unique[value_column].bfill())

    return gaps.reset_index(drop=True)

In [ ]:
hourly_gaps_pm2_5 = detect_hourly_gaps(df_hourly_no_outliers, value_column="pm2_5")
hourly_gaps_temperature = detect_hourly_gaps(df_hourly_no_outliers, value_column="air_temperature")
hourly_gaps_humidity = detect_hourly_gaps(df_hourly_no_outliers, value_column="air_humidity")

# Output found gaps
print(hourly_gaps_pm2_5.head())

## Parameter Correlation Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Select only numeric columns
numeric_columns = ["pm2_5", "air_temperature", "air_humidity"]
df_corr = df_hourly_no_outliers[numeric_columns].corr()

# Visualization using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(df_corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation matrix of hourly data")
plt.show()

In [ ]:
# Visualize PM2.5 relationship with temperature and humidity

# PM2.5 vs Temperature scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_hourly_no_outliers["air_temperature"], y=df_hourly_no_outliers["pm2_5"], alpha=0.5)
plt.xlabel("Temperature, °C")
plt.ylabel("PM2.5, µg/m³")
plt.title("PM2.5 vs Temperature")
plt.show()

# PM2.5 vs Humidity scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_hourly_no_outliers["air_humidity"], y=df_hourly_no_outliers["pm2_5"], alpha=0.5)
plt.xlabel("Humidity, %")
plt.ylabel("PM2.5, µg/m³")
plt.title("PM2.5 vs Humidity")
plt.show()

In [ ]:
# Plot PM2.5 distribution at different temperatures and humidity levels

df_hourly_no_outliers["temp_bins"] = pd.cut(df_hourly_no_outliers["air_temperature"], bins=10)
df_hourly_no_outliers["hum_bins"] = pd.cut(df_hourly_no_outliers["air_humidity"], bins=10)

plt.figure(figsize=(10, 6))
sns.boxplot(x=df_hourly_no_outliers["temp_bins"], y=df_hourly_no_outliers["pm2_5"])
plt.xticks(rotation=45)
plt.xlabel("Temperature ranges")
plt.ylabel("PM2.5")
plt.title("PM2.5 distribution by temperature")
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x=df_hourly_no_outliers["hum_bins"], y=df_hourly_no_outliers["pm2_5"])
plt.xticks(rotation=45)
plt.xlabel("Humidity ranges")
plt.ylabel("PM2.5")
plt.title("PM2.5 distribution by humidity")
plt.show()

## Weather Data Import
1 Data import – Load CSV with parameters: time, temperature, pressure, humidity, wind direction and speed, visibility.

2 Time conversion – Convert time to datetime format.

3 Numeric data cleaning – Convert T, P0, P, U, Ff, VV to float, replace gaps with median.

4 Categorical variable processing – Encode DD (wind direction) with numbers, create decoding dictionary.

5 Visibility cleaning (VV) – Remove text part from values ("10.0 and more" → 10.0).

time (time), T (temperature), P0 (station level pressure), P (sea level adjusted pressure), U (humidity), DD (wind direction), Ff (wind speed), VV (visibility).

In [ ]:
import pandas as pd

# File path
file_path = "weather_archive_utf8.csv"

# Select only needed columns (remove "c")
columns_to_import = ["time", "T", "P0", "P", "U", "DD", "Ff", "VV"]

# Data import
df_weather = pd.read_csv(file_path, delimiter=";", usecols=columns_to_import, encoding="utf-8")

# Convert "time" column to datetime
df_weather["time"] = pd.to_datetime(df_weather["time"], format="%d.%m.%Y %H:%M")

# Process numeric data, replacing commas with dots (if any)
numeric_columns = ["T", "P0", "P", "U", "Ff"]
df_weather[numeric_columns] = df_weather[numeric_columns].replace(",", ".", regex=True).astype(float)

# Fill missing values with median (if any)
df_weather[numeric_columns] = df_weather[numeric_columns].fillna(df_weather[numeric_columns].median())

# Categorical variables: wind direction (DD) → label encoding
df_weather["DD"] = df_weather["DD"].astype(str)
wind_mapping = {direction: idx for idx, direction in enumerate(df_weather["DD"].unique())}
df_weather["DD"] = df_weather["DD"].map(wind_mapping)

# **Process VV (visibility)**
# Keep only number, replace "10.0 and more" with 10.0
df_weather["VV"] = df_weather["VV"].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float)

# Check result
print(df_weather.info())
print(df_weather.head())

# Dictionary for decoding wind direction codes
print("\nEncoded wind directions:")
print(wind_mapping)






In [ ]:
import pandas as pd

# Merge dataframes
# Output original DataFrame structure


df_merged = df_hourly_no_outliers.copy()
df_merged = df_merged.drop(columns=["temp_bins", "hum_bins"], errors="ignore")
print("Original df_merged DataFrame structure:")
print(df_merged.info())

# Convert time column in df_weather to datetime format (if not done yet)
df_weather["time"] = pd.to_datetime(df_weather["time"])

# Convert date column in df_merged to datetime format (if not done yet)
df_merged["date"] = pd.to_datetime(df_merged["date"])

# Filter values where minutes = 00 in df_weather (full hours)
df_weather_hourly = df_weather[df_weather["time"].dt.minute == 0]

# Filter values where minutes = 30 in df_weather (backup values)
df_weather_half_hour = df_weather[df_weather["time"].dt.minute == 30]

# Merge by exact hourly correspondence (00 minutes)
df_merged = pd.merge(
    df_merged, 
    df_weather_hourly, 
    left_on="date", 
    right_on="time", 
    how="left", 
    suffixes=("", "_hourly")  # Add suffix for columns from df_weather_hourly
)

# Remove extra time column (after first merge)
df_merged.drop(columns=["time"], inplace=True)

# Merge with 30-minute data (if 00-minute data is missing)
df_merged = pd.merge(
    df_merged, 
    df_weather_half_hour, 
    left_on="date", 
    right_on="time", 
    how="left", 
    suffixes=("", "_half_hour")  # Add suffix for columns from df_weather_half_hour
)

# Fill gaps with 30-minute values
weather_cols = ["T", "P0", "P", "U", "DD", "Ff", "VV"]
for col in weather_cols:
    df_merged[col] = df_merged[col].fillna(df_merged[f"{col}_half_hour"])

# Remove temporary columns and duplicates
df_merged.drop(columns=["time"] + [f"{col}_half_hour" for col in weather_cols], inplace=True, errors="ignore")

# Output final DataFrame structure
print("Merged DataFrame structure:")
print(df_merged.info())

# Show first rows
print(df_merged.head())






In [ ]:
plot_hourly_param_completeness(df_merged, parameter="P")

## Linear Interpolation of Short Gaps (no more than 2 hours)

In [ ]:
import pandas as pd
import numpy as np

def interpolate_short_gaps(df, value_column, time_column="date", max_gap=1):
    """
    Interpolates only short gaps (no more than max_gap hours).
    
    :param df: DataFrame with data
    :param value_column: Name of data column for interpolation
    :param time_column: Name of time column
    :param max_gap: Maximum gap length for interpolation (in hours)
    :return: DataFrame with interpolated values (only for short gaps)
    """
    df = df.copy()
    df[time_column] = pd.to_datetime(df[time_column])  # Convert to datetime
    df = df.sort_values(time_column)  # Sort by time

    # Define gaps
    df["gap"] = df[value_column].isna()

    # Calculate gap duration
    df["gap_group"] = (df["gap"] != df["gap"].shift()).cumsum()  # Group gaps
    gap_sizes = df.groupby("gap_group")["gap"].transform("sum")  # Size of each gap

    # Keep only short gaps
    df[value_column] = df[value_column].interpolate(method="linear", limit=max_gap)

    # If gap size is larger than max_gap → keep NaN
    df.loc[gap_sizes > max_gap, value_column] = np.nan

    # Remove helper columns
    df.drop(columns=["gap", "gap_group"], inplace=True)

    return df

# 📌 Apply interpolation only for gaps ≤ 2 hours
df_interpolated = df_merged.copy()
for param in ["pm2_5", "air_temperature", "air_humidity", "T", "P0", "P", "U", "DD","Ff", "VV"]:
    df_interpolated = interpolate_short_gaps(df_interpolated, value_column=param, max_gap=2)

# 📊 Output number of remaining gaps
print("\n📊 Number of gaps AFTER interpolation (only short ≤2 hours):")
print(df_interpolated.isna().sum())

# 🔍 Visualize result (NaN heatmap)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.heatmap(df_interpolated.isna(), cmap="coolwarm", cbar=False)
plt.title("Heatmap of gaps after interpolation (≤2 hours)")
plt.show()


In [ ]:
plot_hourly_param_completeness(df_interpolated, parameter="pm2_5")

In [ ]:
df_interpolated.to_csv("df_data_prepared.csv", index=False, encoding="utf-8")